In [1]:
import numpy as np
from pathlib import Path
import pandas as pd
import regex as re
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
from PyPDF2 import PdfWriter, PdfReader

In [2]:
source_material_dir = Path("/Users/vigji/code/cainsjb/source_material")

#### Renumber pages

In [2]:
fname = source_material_dir / "cains_jawbone.txt"
with open(fname, "r") as f:
    text = f.read()

forced_words = ["Head", "May"]
def extract_ne(quote):
    words = word_tokenize(quote, language="english")
    sents = sent_tokenize(quote, language="english")
    tags = nltk.pos_tag(words)
    tree = nltk.ne_chunk(tags, binary=True)
    interesting_words = [" ".join(i[0] for i in t)
        for t in tree if hasattr(t, "label") and t.label() == "NE"] + [w for w in words if w in forced_words]
    
    interesting_sents = {k: [] for k in interesting_words}
    for word in set(interesting_words):
        for s in sents:
            if word in s:
                interesting_sents[word].append(s)
    return interesting_words, interesting_sents
    #for w in words

pages = text.split("________________")

# Extract all name entities:
all_ne = [extract_ne(page) for page in pages]

# Put together occurrencies from all pages:
pages_dict = dict()
for i, (words, sents) in enumerate(all_ne):
    for word in words:
        if word in pages_dict.keys():
            if sents[word] not in pages_dict[word]["fragments"]:
                pages_dict[word]["pages"].append(i)
                pages_dict[word]["fragments"].append(sents[word])
        else:
            pages_dict[word] = dict(pages=[i], fragments=[sents[word]])
            
k = 0        


tosave = ""
for i, page in enumerate(pages):
    tosave += f"------------------\n\n[{i + 1}]\n" + page + "\n\n\n"
    
with open(fname.parent / f"{fname.stem}_renumbered.txt", "w") as f:
    f.write(tosave)

#### Split pdf

In [3]:
inputpdf = PdfReader(open(source_material_dir / "cains_jawbone.pdf", "rb"))

for i in range(len(inputpdf.pages)):
    output = PdfWriter()
    output.add_page(inputpdf.pages[i])
    with open(source_material_dir / "pages" / f"page_{i}.pdf", "wb") as outputStream:
        output.write(outputStream)

In [ ]:
pages_dict["Aquarius"]

In [ ]:
pages_dict["Alexander"]

In [ ]:
pages_dict["Head"]

In [ ]:
pages_dict["May"]

In [ ]:
pages[57]

In [62]:
words = word_tokenize(page, language="english")
sentencies = sent_tokenize(page, language="english")
tags = nltk.pos_tag(words)
tree = nltk.ne_chunk(tags, binary=True)

In [ ]:
for k in sorted(pages_dict):
    pgs, sents = pages_dict[k]["pages"], pages_dict[k]["fragments"]
    if len(set(pgs)) > 1:
        print("---------")
        print(k)
        print([p+1 for p in pgs])
        for s in sents:
            print(f"--{s}")

In [ ]:
pages[85]

In [ ]:
text.lower().index("fox")

In [ ]:
text[3000:5300]

In [ ]:
for k in sorted(pages_dict):
    pages, sents = pages_dict[k]["pages"], pages_dict[k]["fragments"]
    #if len(set(pages)) > 1:
    print("---------")
    print(k)
    print([p+1 for p in pages])
    for s in sents:
        print(f"--{s}")

In [ ]:
for word, pages in pages_dict.items():
    print(word, pages)